<a href="https://colab.research.google.com/github/EdBerg21/AI-Professional-Prompts/blob/main/DONE_NATIONAL_UNIFIED_rag_with_mistral_7b_query_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# bitsandbytes is required for 4-bit quantization by transformers.AutoModelForCausalLM
# sentence-transformers is required by model wrapper langchain.embeddings.HuggingFaceEmbeddings
# faiss-gpu is required by embeddings db langchain.vectorstores.FAISS
!pip install -q langchain bitsandbytes sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00


In [5]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00


In [5]:
from torch import cuda, bfloat16
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)
assert 'cuda' in device

cpu


AssertionError: 

In [31]:
from pathlib import Path
from pprint import pprint
import sys
from timeit import default_timer

import numpy as np
import torch
from torch.nn import functional as F
import transformers
from transformers import AutoTokenizer
from transformers import top_k_top_p_filtering
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.schema.document import Document

In [30]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
model_path = "/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/9ab9e76e2b09f9f29ea2d56aa5bd139e4445c59e"

# Model
## Files

In [28]:
def sizeof_fmt(num, suffix="B"):
    for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
        if abs(num) < 1024.0:
            return f"{num:5.1f} | {unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

path = Path(model_path)
files = list(path.iterdir())
files = sorted(files, key=lambda f: -f.stat().st_size)
for file in files:
    print(f'{file.as_posix() : <80.80}| {sizeof_fmt(file.stat().st_size)}')

/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|   9.3 | GiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|   4.2 | GiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|   1.7 | MiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho| 481.9 | KiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|  24.5 | KiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|   1.4 | KiB
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho| 571.0 | B
/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapsho|  72.0 | B


## Tokenizer
Load pretrained tokenizer.

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

## Model

### Mistral Config

In [26]:
# the config shows us we need to instantiate huggingface's AutoModelForCausalLM
model_config = transformers.AutoConfig.from_pretrained(model_path)
model_config

MistralConfig {
  "_name_or_path": "/root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/9ab9e76e2b09f9f29ea2d56aa5bd139e4445c59e",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 32000
}

### 4-bit Quantization

<div class="alert alert-block alert-info">
<b>4-bit Quantization</b><br><br> Quantization in the context of deep learning is the process of constraining the number of bits that represent the weights and biases of the model - Weights and Biases numbers that we need in backpropagation. In 4-bit quantization, each weight or bias is represented using only 4 bits as opposed to the typical 32 bits used in single-precision floating-point format (float32). The primary advantage of using 4-bit quantization is the reduction in model size and memory usage. <br><br>

See https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2

</div>

In [25]:

import transformers
from bitsandbytes import bfloat16  # Assuming bfloat16 is part of the bitsandbytes package

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

ImportError: cannot import name 'bfloat16' from 'bitsandbytes' (/usr/local/lib/python3.10/dist-packages/bitsandbytes/__init__.py)

In [23]:
# requires bitsandbytes package to be installed
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
  #  bnb_4bit_compute_dtype=bfloat16
)

### Pretrained Model

In [5]:
!pip install transformers

In [7]:
import transformers

In [20]:
!pip install accelerate

In [32]:
#t0 = default_timer()
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    torch_dtype = torch.bfloat16,
   # device_map="auto",  # {"": 1},  # "cuda:0",
     device_map='auto',
     load_in_4bit=True
)
print(default_timer(), 'sec.')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1564.02500114 sec.


<div class="alert alert-block alert-warning">
<b>Quantized Model</b><br><br> We currently can't save a model that has been quantized with BitsAndBytes <br><br>

See Pull Request https://github.com/huggingface/transformers/pull/26037

</div>

In [33]:
print('Architecture\n')
model

Architecture



MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [34]:
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

ModuleNotFoundError: No module named 'pynvml'

In [35]:
print(f'The model has {model.num_parameters():,} parameters.')
print(f'The Model runs on {model.device}')
print_gpu_utilization()

The model has 7,241,732,096 parameters.
The Model runs on cuda:0


NameError: name 'print_gpu_utilization' is not defined

In [36]:
print('Model (without head) \n\n')
model.model

Model (without head) 




MistralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
    )
  )
  (norm): MistralRMSNorm()
)

# Test Model

In [39]:
query = "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words."
messages = [{
    "role":"user",
    "content": query
}]
model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')

In [40]:
# Setting `pad_token_id` to `eos_token_id` for open-ended generation.
generated_ids = model.generate(
    model_inputs,
    max_new_tokens = 7500,
    do_sample = True,
    pad_token_id = tokenizer.eos_token_id

)
print('Generated IDs:', generated_ids)

Generated IDs: tensor([[    1,   733, 16289, 28793,  5919,  7282,   767,   349,   272,  3885,
           302,   272,  7085,  2962, 28723, 16104,   776,   264,  7526, 28723,
         10339,   378,   297, 28705, 28740, 28734, 28734,  3085, 28723,   733,
         28748, 16289, 28793,   415,  3885,   302,   272,  7085,  2962,   349,
           396,  9558,  1951,   297,   690,   272,  4951,   302,   272,  2969,
          3543,   882,  1588,   264,  8666,   298,  8463, 28725,  9566,   297,
          2935,  4624, 28723,  6213,   272,  8666, 28725,   272,  4951,  5312,
           396,  4009,   356,   272,  4644,   302,   272,  5878,   304,  3342,
           274,   272,  1868,  1665,   302,   272,  8725, 28725,  4282,  4908,
         28725,   304,   799, 20902,  4382, 28723,   415,  8666,   349,  5035,
           298,  3084,   264,  3081,   304,  3078,   864, 23094,   302,   272,
          4951, 28742, 28713, 28231,   304,  6400,   354,   272,  3524,   879,
         28725,   304,   378,  2608, 

In [41]:
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print('Decoded:', decoded)

Decoded: ["[INST] Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words. [/INST] The State of the Union address is an annual event in which the president of the United States delivers a speech to Congress, typically in early January. During the speech, the president provides an update on the condition of the nation and discusses the current state of the economy, national security, and other pressing issues. The speech is meant to provide a clear and concise overview of the president's priorities and plans for the coming year, and it often includes calls to action for Congress."]


# RAG

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/RAG_schema.svg/1024px-RAG_schema.svg.png)

In [42]:
# Wrap tokenizer and model with pipeline
pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    # temperature=0.1,
    max_new_tokens=150,
    # repetition_penalty=1.1
    return_full_text=False,
#     torch_dtype=torch.float16,
    pad_token_id = tokenizer.eos_token_id,  # open-end generation (and suppressing warning...)
)

In [43]:
# Wrap pipeline with langchain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline)

# RAG - State of the Union 2023

In [46]:
loader = TextLoader("/content/biden-sotu-2023-autogenerated-transcript.txt",
                    encoding="utf8")
documents = loader.load()
print(len(documents), 'Document')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
print(len(all_splits), 'Chunks')

1 Document
54 Chunks


In [47]:
embeddings_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings_model_kwargs = {"device": "cuda"}

# HuggingFaceEmbeddings is a langchain class
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name,
                                   model_kwargs=embeddings_model_kwargs)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [48]:
db = FAISS.from_documents(all_splits, embeddings)
retriever = db.as_retriever()  # langchain_core.vectorstores.VectorStoreRetriever

In [49]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [50]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words."
print(f"Query: {query}\n")
result = qa.run(query)
print("\nResult: ", result)

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

Result:   The main topics in the State of the Union in 2023 were the economy, healthcare, climate change, and national security. The President spoke about the strong state of the union, the importance of expanding healthcare coverage, the need for climate change action, and the need for increased national security measures. The President also spoke about the importance of bipartisanship and working together to address these issues. The President also mentioned the need for increased funding for mental health care and the need to hold social media companies accountable for their actions.


In [51]:
relevant_documents = retriever.get_relevant_documents(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(relevant_documents)}\n")
for doc in relevant_documents:
    print(doc.page_content[:450], '...\n')

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words.
Retrieved documents: 4

of light over dark

hope over fear you never do the

stability over chaos

we have to see each other not as enemies

but as fellow Americans

we're good people the only nation in the

world built on an idea the only one

other nations

are defined by geography ethnicity

[Applause]

but we're the only nation based on an

idea

that all of us

every one of us is created equal in the

image of God

a nation that stands as a beacon of the

world

a  ...

producing jobs I will veto it

look

I'm pleased to say the more Americans

helpful to have health insurance now

than ever in history a record 16 million

people are enrolled in the Affordable

Care Act and thanks

thanks for the law I signed last year

saving millions of saving 800 a year on

their premiums by the way that law was

written

and the benefit expires in 2025.

so my plea to some of you at least in

# RAG - Static Wikipedia Article
https://en.wikipedia.org/wiki/Shane_MacGowan

In [52]:
article = """

Shane Patrick Lysaght MacGowan (25 December 1957 – 30 November 2023) was an Irish singer and songwriter who was best known as the lead singer and songwriter of Celtic punk band the Pogues. Many of his songs are influenced by Irish nationalism, Irish history, the experiences of the Irish diaspora (particularly in England and the United States), and London life in general.[1]

Born in Kent, England, to Irish parents, MacGowan joined the punk band The Nipple Erectors before founding the Pogues in 1982. He drew upon his Irish heritage when founding The Pogues and changed his early punk style for a more traditional sound with tutoring from his extended family. Between 1985 and 1987, he co-wrote the Christmas hit "Fairytale of New York", which he performed with Kirsty MacColl. In the following years MacGowan and The Pogues released several albums.

After The Pogues fired MacGowan for unprofessional behaviour mid-tour, he formed a new band, Shane MacGowan and The Popes, with whom he recorded two studio albums. In 2001, MacGowan rejoined The Pogues for reunion shows and remained with the group until 2014. MacGowan produced his own solo material and collaborated with artists such as Joe Strummer, Nick Cave, Steve Earle, Sinéad O'Connor, and Ronnie Drew. Throughout his life MacGowan suffered physically from years of binge drinking.

Early life
MacGowan was born on 25 December 1957[2] in Pembury, Kent,[3] the son of Irish immigrants. His father was from Dublin and his mother was from Tipperary. His mother, Therese, worked as a typist at a convent[4] and had previously been a singer, traditional Irish dancer, and model. His father, Maurice, came from a middle-class background and worked in the offices of department store C&A; he was, in his own words, a "local roustabout". MacGowan's younger sister, Siobhan MacGowan, became a journalist, writer, and songwriter. He was born in England, but raised in Tipperary in Ireland until the age of 6.[5]

MacGowan lived in many parts of southeast England such as Brighton, London, and the home counties, and attended an English public school. In 1971 he left Holmewood House preparatory school in Langton Green, Kent, with a scholarship for Westminster School.[6] He was found in possession of drugs and expelled in his second year.[7] He was first publicly noted in 1976 at a concert by London punk rock band The Clash, where his earlobe was damaged by future Mo-dettes bassist Jane Crockford. A photographer took a picture of him covered in blood, which made the local papers with the headline "Cannibalism at Clash Gig".[8][9][10] Shortly after this, he later joined punk band the Nipple Erectors (later known as 'The Nips'), which featured bassist Shanne Bradley.[11]

Career
1982–1991: Leading the Pogues
MacGowan drew upon his Irish heritage when founding the Pogues and changed his early punk style for a more traditional sound with tutoring from his extended family. Many of his songs are influenced by Irish nationalism, Irish history, the experiences of the Irish diaspora (particularly in England and the United States), and London life in general.[1] These influences are documented in the biography Rake at the Gates of Hell: Shane MacGowan in Context. He often cited the 19th-century Irish poet James Clarence Mangan and playwright Brendan Behan as influences.[1] Their most critically acclaimed album was If I Should Fall from Grace with God (1988), which also marked the high point of the band's commercial success. Between 1985 and 1987, MacGowan co-wrote "Fairytale of New York", which he performed with Kirsty MacColl, and remains a perennial Christmas favourite. In 2004, 2005 and 2006, it was voted favourite Christmas song in a poll by music video channel VH1.[12] In the following years MacGowan and the Pogues released several albums.[1][13] In 1988 he co-wrote Streets of Sorrow/Birmingham Six, a song by the Pogues which proved highly controversial, banned on British TV and radio.[14]

1992–2005: Shane MacGowan and the Popes
After the Pogues fired MacGowan for unprofessional behaviour mid-tour, he formed a new band, Shane MacGowan and the Popes, recording two studio albums, a live album, three tracks on the Popes Outlaw Heaven (2010) and a live DVD, and touring internationally. In 1997, MacGowan appeared on Lou Reed's "Perfect Day", covered by numerous artists in aid of Children in Need. It was the UK's number one single for three weeks, in two separate spells.[15] Selling over a million copies, the record contributed £2,125,000 to the charity's highest fundraising total in six years.[16] From December 2003 up to May 2005, Shane MacGowan and the Popes toured extensively in the UK, Ireland and Europe.[17]

2001–2014: Return to the Pogues
A colourful image of Shane
MacGowan depicted in the painting Boy from the County Hell by Brian Whelan
The Pogues and MacGowan reformed for a sell-out tour in 2001 and each year from 2004 to 2009 for further tours, including headline slots at Guilfest in England and the Azkena Rock Festival in the Basque Country. In May 2005, MacGowan rejoined the Pogues permanently.[17] That same year, the Pogues re-released "Fairytale of New York" to raise funds for the Justice For Kirsty Campaign and Crisis at Christmas. The single was the best-selling Christmas-themed single of 2005, reaching number 3 in the UK Charts that year.[18]

In 2006, he was voted 50th in the NME Rock Heroes List.[19][20] He was seen many times with the Libertines and Babyshambles singer Pete Doherty;[21] on occasions MacGowan joined Babyshambles on stage. Other famous friends included Johnny Depp, who appeared in the video for "That Woman's Got Me Drinking",[22] and Joe Strummer, who referred to MacGowan as "one of the best writers of the century" in an interview featured on the videogram release "Live at the Town and Country Club" from 1988. Strummer occasionally joined MacGowan and the Pogues on stage (and briefly replaced MacGowan as lead vocalist after his sacking from the band).[23] He also worked with Nick Cave and joined him on stage.[24]

About his future with the Pogues, in a 24 December 2015 interview with Vice magazine,[25] when the interviewer asked whether the band were still active, MacGowan said: "We're not, no," saying that, since their 2001 reunion happened, "I went back with [the] Pogues and we grew to hate each other all over again," adding: "I don't hate the band at all — they're friends. I like them a lot. We were friends for years before we joined the band. We just got a bit sick of each other. We're friends as long as we don't tour together. I've done a hell of a lot of touring. I've had enough of it."[26]

2010–2011: the Shane Gang
In 2010, MacGowan played impromptu shows in Dublin with a new five-piece backing band named the Shane Gang, including In Tua Nua rhythm section Paul Byrne (drums) and Jack Dublin (bass), with manager Joey Cashman on whistle. In November 2010, this line up went to Lanzarote to record a new album.[27][28] MacGowan and the Shane Gang performed at the Red Hand Rocks music festival in the Patrician Hall, Carrickmore County Tyrone in June 2011.[29]

2011–2023
MacGowan made a return to stage on 13 June 2019 at the RDS Arena in Dublin as a guest for Chrissie Hynde and the Pretenders.[30]

Following on from the success of Feis Liverpool 2018's finale in which he was joined by names such as Imelda May, Paddy Moloney,[31] Albert Hammond Jr and many more, MacGowan was announced to appear on 7 July alongside a host of guests for the Feis Liverpool 2019's finale. However the event was ultimately cancelled due to a lack of ticket sales and funding issues. Feis Liverpool is the UK's largest celebration of Irish music and culture.[32]

In 2020, MacGowan reportedly returned to the studio to record several new songs with the Irish indie band Cronin led by brothers Johnny and Mick Cronin.[33]

Media and charity work
MacGowan appeared in an episode of Fair City, shown on 28 December 2008.[34] In 2009, MacGowan starred in the RTÉ reality show Victoria and Shane Grow Their Own, as he and his wife Victoria Mary Clarke endeavoured to grow their own food in their own garden.[35]

In 2010, MacGowan offered a piece of unusual art to the Irish Society for the Prevention of Cruelty to Children (ISPCC) to auction off to support their services to children: a drawing on a living room door.[36] It ended up earning €1,602 for the charity.[37]

Personal life

MacGowan performing in 2010
Relationships
On 26 November 2018, after a decades-long relationship and subsequent 11-year engagement, MacGowan married Irish journalist Victoria Mary Clarke in Copenhagen. They resided in Dublin.[38] MacGowan was a Roman Catholic, describing himself as "a free-thinking religious fanatic" who also prayed to Buddha. As an adolescent, he considered the priesthood.[39]

Addictions
In 2001, Sinéad O'Connor reported MacGowan to the police in London for drug possession, in what she said was an attempt to discourage him from using heroin.[40] At first furious, MacGowan later expressed gratitude towards O'Connor and claimed that the incident helped him kick his heroin habit.[41]

MacGowan suffered physically from years of binge drinking.[42] He often performed onstage and gave interviews while drunk. In 2004, on the BBC TV political magazine programme This Week, he gave incoherent and slurred answers to questions from Janet Street-Porter about the public smoking ban in Ireland.[43] MacGowan began drinking at age five, when his family gave him Guinness to help him sleep, and his father frequently took him to the local pub while he drank with his friends.[44]

In 2016, Clarke revealed to the press that MacGowan was sober "for the first time in years." She explained that the origins of MacGowan's drinking problem stemmed from several years of "singing in bars and clubs and other venues where people go to drink and have fun" and that "his whole career has revolved around it and, indeed, been both enhanced and simultaneously inhibited by it". She said that his drinking was not a problem for many years but "went from being just a normal part of life" to becoming very unhealthy, a circumstance made much worse due to the introduction of hard drugs such as heroin. She explained that a serious bout with pneumonia, compounded by an excruciatingly painful hip injury which required a long stay in the hospital, was ultimately responsible for his sobriety. The lengthy hospital stay required a total detox, and MacGowan's sobriety continued after he got home.[45]

Politics
Having grown up in an Irish republican family, MacGowan said in 2015 that he regretted not joining the IRA. In a filmed interview he said, "I was ashamed I didn't have the guts to join the IRA, and The Pogues was my way of overcoming that".[46][47] The central figure in his 1997 song "Paddy Public Enemy No. 1" is based on ex-INLA leader Dominic McGlinchey. Asked his opinion of McGlinchey, MacGowan said "he was a great man".[48] He also counted Sinn Féin leader Gerry Adams as a friend, according to his most recent biography.[49]

Health
MacGowan used a wheelchair following a fall as he was leaving a Dublin studio in the summer of 2015, which fractured his pelvis.[33] He said in an interview with Vice later that year, "It was a fall and I fell the wrong way. I broke my pelvis, which is the worst thing you can do. I'm lame in one leg, I can't walk around the room without a crutch. I am getting better, but it's taking a very long time. It's the longest I've ever taken to recover from an injury. And I've had a lot of injuries."[50] He continued to use a wheelchair until his death in 2023.[51][52]

MacGowan was long known for having very bad teeth; he lost the last of his natural teeth sometime around 2008. In 2015, he had a new set of teeth, with one gold tooth, fitted in a nine-hour procedure. These were retained by eight titanium implants in his jaws. The procedure was the subject of the hour-long television programme Shane MacGowan: A Wreck Reborn. The dental surgeon who carried out the procedure commented that MacGowan had recorded most of his great works while he still had some teeth: "We've effectively re-tuned his instrument and that will be an ongoing process."[26][53]

MacGowan was hospitalised for an infection on 6 December 2022.[54][55] He was diagnosed with viral encephalitis.[56]

Final illness and death
It was reported in July 2023 that MacGowan was hospitalised in the intensive care unit (ICU).[57] Following treatment for an infection, he was discharged in November.[58] He made his last public statement on November 16, 2023, complimenting Travis Kelce's cover of "Fairytale of New York".[59] MacGowan died, at home with his wife by his side, on 30 November 2023.[60][61][62]

Following MacGowan's death, Michael D. Higgins, President of Ireland, said: "Shane will be remembered as one of music’s greatest lyricists. So many of his songs would be perfectly crafted poems, if that would not have deprived us of the opportunity to hear him sing them. The genius of Shane’s contribution includes the fact that his songs capture within them, as Shane would put it, the measure of our dreams - of so many worlds, and particularly those of love, of the emigrant experience and of facing the challenges of that experience with authenticity and courage, and of living and seeing the sides of life that so many turn away from."[63]

Autobiography and biography
In 2001, MacGowan coauthored the autobiographical book A Drink with Shane MacGowan with his then partner, later wife, Victoria Mary Clarke.[64]

Aside from Rake at the Gates of Hell: Shane MacGowan in Context, which covered up to partway through his musical career, MacGowan was the subject of a 2015 biography, A Furious Devotion: The Life of Shane MacGowan.[65] He was also the subject of several books and paintings. In 2000, Tim Bradford used the title Is Shane MacGowan Still Alive? for a humorous book about Ireland and Irish culture.[66] Shaman Shane — The Wounded Healer by Stephan Martin brands Shane as a latter-day London-Irish spirit-raiser and exorcist. This commentary is found in the book Myth of Return — The Paintings of Brian Whelan and Collected Commentaries. London Irish artist Brian Whelan has painted MacGowan (for example Boy From The County Hell), his works are featured on MacGowan's official website, and is also the illustrator of The Popes’ Outlaw Heaven cover.[67]

Honours and awards

MacGowan receiving a Lifetime Achievement Award from President of Ireland, Michael D. Higgins, in the National Concert Hall, Dublin, on 15 November 2018
In January 2018, MacGowan was honoured with a concert gala to celebrate his 60th birthday at the National Concert Hall in Dublin, where he was presented the Lifetime Achievement Award by Irish President Michael D. Higgins.[68] He also won the 2018 Ivor Novello Inspiration Award.[69]

"""

In [53]:
# Mistral (unlike LLama2) has problems with [...] annotations. Let's remove them
import re
PATTERN = '\[[^()]*\]'
article = re.sub(PATTERN, "", article)

In [54]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = [Document(page_content=x) for x in text_splitter.split_text(article)]
print(f'Split into {len(documents)} Documents')

Split into 6 Documents


In [55]:
db = FAISS.from_documents(documents, embeddings)
retriever = db.as_retriever()  # langchain_core.vectorstores.VectorStoreRetriever

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [56]:
query = "Please summarize the article in one sentence."
print(f"Query: {query}\n")
result = qa.run(query)
print("\nResult: ", result, '\n\n')

relevant_documents = retriever.get_relevant_documents(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(relevant_documents)}\n")
for doc in relevant_documents:
    print(doc.page_content[:450], '...\n')

Query: Please summarize the article in one sentence.



> Entering new RetrievalQA chain...

> Finished chain.

Result:   Shane MacGowan, the lead singer and songwriter of the Pogues, formed a new band, Shane MacGowan and the Popes, after being fired from the Pogues, and later joined punk band the Nipple Erectors. 


Query: Please summarize the article in one sentence.
Retrieved documents: 4

In 2010, MacGowan offered a piece of unusual art to the Irish Society for the Prevention of Cruelty to Children (ISPCC) to auction off to support their services to children: a drawing on a living room door. ...

1992–2005: Shane MacGowan and the Popes
After the Pogues fired MacGowan for unprofessional behaviour mid-tour, he formed a new band, Shane MacGowan and the Popes, recording two studio albums, a live album, three tracks on the Popes Outlaw Heaven (2010) and a live DVD, and touring internationally. In 1997, MacGowan appeared on Lou Reed's "Perfect Day", covered by numerous artists in aid of 

# RAG - Fetch Wikipedia Page with requests

Request and parse a random Wikipedia page

In [57]:
import requests

def get_wikipedia_page(page_title):
    endpoint = "https://en.wikipedia.org/w/api.php"
    params = {
    "format": "json",
    "action": "query",
    "prop": "extracts",
    # "exintro": "",
    "explaintext": "",
    "titles": page_title
    }
    response = requests.get(endpoint, params=params)
    data = response.json()
    pages = data["query"]["pages"]
    page_id = list(pages.keys())[0]
    return pages[page_id]["extract"]

In [58]:
def get_chain(article: str):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    documents = [Document(page_content=x) for x in text_splitter.split_text(article)]
    print(f'Split into {len(documents)} Documents')

    db = FAISS.from_documents(documents, embeddings)
    retriever = db.as_retriever()  # langchain_core.vectorstores.VectorStoreRetriever

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        verbose=True
    )
    return qa, retriever

In [59]:
# page_title = "Electoral system of Germany"
page_title = "Tuberculosis"
wikipedia_page = get_wikipedia_page(page_title)
qa, retriever = get_chain(wikipedia_page)

Split into 53 Documents


In [60]:
#query = "Please give a summary in 100 words."
query = "How many people die from Tuberculosis?"
print(f"Query: {query}\n")
result = qa.run(query)
print("\nResult: ", result, '\n\n')

relevant_documents = retriever.get_relevant_documents(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(relevant_documents)}\n")
for doc in relevant_documents:
    print(doc.page_content[:450], '...\n')

Query: How many people die from Tuberculosis?



> Entering new RetrievalQA chain...

> Finished chain.

Result:   In 2010, 1.20–1.45 million deaths occurred due to Tuberculosis. 


Query: How many people die from Tuberculosis?
Retrieved documents: 4

== See also ==
List of deaths due to tuberculosis


== Notes ==


== References ==


== External links == ...

== Epidemiology ==
Roughly one-quarter of the world's population has been infected with M. tuberculosis, with new infections occurring in about 1% of the population each year. However, most infections with M. tuberculosis do not cause disease, and 90–95% of infections remain asymptomatic. In 2012, an estimated 8.6 million chronic cases were active. In 2010, 8.8 million new cases of tuberculosis were diagnosed, and 1.20–1.45 million deaths occurre ...

Tuberculosis (TB), also known colloquially as the "white death", or historically as consumption, is an infectious disease usually caused by Mycobacterium tuberculosis (MTB) bacteria

In [61]:
# page_title = "Electoral system of Germany"
page_title = "Electoral system of Germany"
wikipedia_page = get_wikipedia_page(page_title)
qa, retriever = get_chain(wikipedia_page)

Split into 28 Documents


In [62]:
#query = "Please give a summary in 100 words."
query = "Describe the importance of the First Vote in the German Voting System"
print(f"Query: {query}\n")
result = qa.run(query)
print("\nResult: ", result, '\n\n')

relevant_documents = retriever.get_relevant_documents(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(relevant_documents)}\n")
for doc in relevant_documents:
    print(doc.page_content[:450], '...\n')

Query: Describe the importance of the First Vote in the German Voting System



> Entering new RetrievalQA chain...

> Finished chain.

Result:   The first vote in the German voting system is important because it allows the elector to vote for a direct candidate of their constituency. This personalizes the election and allows the elector to have a direct say in who represents their local area in the Bundestag. The first vote also determines the number of direct mandates in the Bundestag, with each district electing one direct candidate. However, the first vote does not determine the power of the parties in the Bundestag. For each direct mandate in a Bundesland, the party always receives one mandate fewer from the second vote. 


Query: Describe the importance of the First Vote in the German Voting System
Retrieved documents: 4

=== First vote ===
The first vote allows the elector to vote for a direct candidate of their constituency, who applies for a direct mandate in the Bundestag (se

References:
- https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb
- https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore
- https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html